# Final Project for DS5100
## Student: Kyler Halat-Shafer
## User ID: uxt5qb

In [1]:
import pandas as pd
import numpy as np
import math
import os
import random
import seaborn as sns
import unittest

### Important Information about the project:

In this project you write, package, and publish a Python module and accompanying files. The project will implement a simple Monte Carlo simulator using a set of related classes.

#### Requirements

You write three classes that will work in tandem to generate various outcomes.

- A Die class
- A Game class
- An Analyzer class

In addition, you will write unit tests, a scenario script, and documentation for these classes.

Note: You will put all three classes in a single file called montecarlo.py and the unit tests in a file called montecarlo_tests.py. The scenario script will be put into a Jupyter Notebook called montecarlo_demo.ipynb.

### A Die Class

Required scenarios:
- Two headed coin
- 6 sided die
- 26 lettered alphabet

In [48]:
class Die():
    
    def __init__(self, faces):
        self.faces = faces #making the faces the length of the weights
        self.weights = np.ones(len(faces)) #defining the weights 
        self.df = pd.DataFrame({'faces': self.faces, 'weights':self.weights}) # dataframe with both faces and weights
        print(self.df)
        
    def change_the_weight(self,new_face, new_weight):
        
        if new_face not in list(self.df['faces']):
            print('Value did not pass')
        else:
            if (type(new_weight)) not in [int,float]:
                print('Change to integer or float to continue')
            else:
                self.df.loc[self.df.faces == new_face, 'weights'] = new_weight
        
    def roll(self, num_rolls = 1): #this is taking the argument of the number of rolls, defaulting to 1
        results = [] 
        #my_probs = [i/sum(self.weights) for i in self.weights] 
        for i in range(num_rolls): 
            result = self.df.faces.sample(weights = self.df['weights']).values[0] 
            results.append(result)
        return results
    
    def show(self):
        print('The current state faces and weights:') 
        return self.df

In [49]:
d = Die([1,2,3,4,5,6])

   faces  weights
0      1      1.0
1      2      1.0
2      3      1.0
3      4      1.0
4      5      1.0
5      6      1.0


In [50]:
d.change_the_weight(5,6)

In [51]:
d.roll(10)

[5, 5, 6, 3, 5, 5, 5, 5, 5, 5]

In [52]:
d.show()

The current state faces and weights:


,faces,weights
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,6.0
5,6,1.0


In [57]:
d2 = Die([1,2,3,4,5,6])

   faces  weights
0      1      1.0
1      2      1.0
2      3      1.0
3      4      1.0
4      5      1.0
5      6      1.0


In [58]:
d2.change_the_weight(3,6)

In [59]:
d2.roll(10)

[3, 5, 3, 2, 3, 5, 2, 3, 3, 3]

In [60]:
d2.show()

The current state faces and weights:


,faces,weights
0,1,1.0
1,2,1.0
2,3,6.0
3,4,1.0
4,5,1.0
5,6,1.0


Notes:
- to make a private dataframe use two underscores before (__) 

### A Game Class

In [285]:
# Once the number of rolls, number of sides, and its weights are created in the first class, we then take that class 
# bring those 3 pieces of information into this game class, to choose how many dice will then be rolled 

class DieGame():
    
    def __init__(self,dice):
        self.dice = dice #this will call on d and d2 which are objects that I created
        
    def play2(self,n_rolls = 1):
        
        self.roll_results = pd.DataFrame(index=range(1,n_rolls + 1))
        self.roll_results.index.rename('Roll_Number' ,inplace = True)
        n = 0
        for die in self.dice: #for each die in dice
            roll = die.roll(n_rolls) #iterating over the multiple dies in dice, for the amount of rolls 
            self.roll_results[n] = roll #starting as a blank dataframe then adding the roll 
            n = n + 1 #need to define a column with each new die
            
    
    def display(self, wide = True):
        #self.roll_results["id"] = self.roll_results.index
        d1 = self.roll_results.copy()
        d1['id'] = d1.index
        narrowdf = pd.melt(d1, id_vars = ["id"],value_vars = [0,1])
        narrowdf.columns = ['Roll_Number','Die','Face_Value']
        narrowdf.set_index(['Roll_Number','Die'],inplace = True)
        #self.roll_results.drop(['id'],inplace = True)
        
        if wide == True:
            return self.roll_results
        if wide == False:
            return narrowdf
        else:
            return 'Please enter True to return a wide dataframe and False for a narrow dataframe.'

In [198]:
game = DieGame([d,d2])

In [199]:
game.play2(10)

In [200]:
game.roll_results

,0,1
Roll_Number,,
1,5,2
2,2,6
3,1,1
4,1,3
5,5,1
6,5,3
7,5,4
8,5,3
9,2,3


In [201]:
game.display(False)

,,Face_Value
Roll_Number,Die,
1,0,5
2,0,2
3,0,1
4,0,1
5,0,5
6,0,5
7,0,5
8,0,5
9,0,2


### An Analyzer Class

In [284]:
class Analyzer (): 
    
    def __init__(self,game): 
        self.game = game
        game.display().dtypes
    
    def jackpot(self):
        ww = []
        for r in range(len(game.display())):
            if len(set(list(game.display().loc[r+1,:]))) == 1:
                ww.append(True)
            else:
                ww.append(False)
        self.jack = game.display()[ww]
        return len(self.jack)
    
    def combo(self):
        self.comb = pd.DataFrame(game.display().groupby(list(game.display().columns)).size())
        self.comb.columns = ['Number of Instances']
        return self.comb 
    
    def face_counts(self):
        self.gamecross = pd.crosstab(index = list(game.display(False).reset_index()['Roll_Number']), columns = list(game.display(False)['Face_Value']))
        return self.gamecross

In [276]:
analyze1 = Analyzer(game)
analyze1.jackpot()
analyze1.jack

,0,1
Roll_Number,,
3,1,1


In [268]:
list(game.display(False).reset_index()['Roll_Number'])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [277]:
analyze1.combo()

Number of Instances
0 1                     
1 1                    1
  3                    1
2 3                    1
  6                    1
5 1                    1
  2                    1
  3                    3
  4                    1

In [278]:
analyze1.face_counts()

col_0,1,2,3,4,5,6
row_0,,,,,,
1,0,1,0,0,1,0
2,0,1,0,0,0,1
3,2,0,0,0,0,0
4,1,0,1,0,0,0
5,1,0,0,0,1,0
6,0,0,1,0,1,0
7,0,0,0,1,1,0
8,0,0,1,0,1,0
9,0,1,1,0,0,0


In [279]:
analyze1.gamecross

col_0,1,2,3,4,5,6
row_0,,,,,,
1,0,1,0,0,1,0
2,0,1,0,0,0,1
3,2,0,0,0,0,0
4,1,0,1,0,0,0
5,1,0,0,0,1,0
6,0,0,1,0,1,0
7,0,0,0,1,1,0
8,0,0,1,0,1,0
9,0,1,1,0,0,0


In [235]:
list(a.index)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [239]:
list(a[0])

[5, 2, 1, 1, 5, 5, 5, 5, 2, 5]

### Unit Testing

In [ ]:
#import unittest
#import pandas as pd
#import numpy as np
#from module import Die
#from module import DieGame
#from module import Analyzer

In [ ]:
# Each method in each class should have unit tests to test if the methods receive the correct inputs and return valid outputs

class DieGameTestSuite(unittest.TestCase):

    n_sides = 6
    weights = [1,1,1,1,1,1] #This is producing a fair game, where each weight is equal to 1 
    df = pd.DataFrame({'n_sides':[], 'weights':[]}) # dataframe with both n_sides and weights
    df2 = pd.DataFrame({'num_rolls':[], 'num_dice':[]})
    
    def test_changing_weight(self): 
        testing = Die()
    #Is the face that passed in the array of weights?

    def test_roll(self):
        testing = Die()
    # Does it take in how many times the die is to be rolled? Does this return a list of outcomes? 
    
    def test_show(self):
        testing = Die()
    # Does this show the current set of faces and weights? 
    # use a regular die

    def test_play(self): 
        testing = DieGame()
    # Does it create a table where the columns are roll number, the die number is the row, and the element is the face?
                        
    def test_display(self):
        testing = DieGame()
    # Is a narrow and a wide dataframe able to be created?
    # use a regular die
        
    def test_jackpot(self):
        testing = Analyzer()
    # Check the dataframe that is created if all face_values are identical
    # use a game where both face values on the die are the same to see if it works 
    
    def test_combo(self):
        testing = Analyzer()
    # Check the dataframe that is created if all paired face_values are unique
    # use a regular die
    
    def test_face_count(self):
        testing = Analyzer()
    # Check the dataframe that your face_count is saved
    # use a regular die